# Proyecto SQL
### Objetivos del estudio
Sacaremos información de bases de datos de libros, editoriales, autores, calificaciones de clientes y reseñas de libros para generar una propuesta de valor para un nuevo startup para los amantes de los libros.

### Exploración de tablas

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
db_config = {
 'user': 'practicum_student', # username
 'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7', # password
 'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
 'port': 5432, # connection port
 'db': 'data-analyst-final-project-db' # the name of the database
 }
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],db_config['pwd'],db_config['host'],db_config['port'],db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
query = """ 
SELECT *
FROM books
LIMIT 5;
"""

books = pd.io.sql.read_sql(query, con = engine)
print(books)

   book_id  author_id                                              title  \
0        1        546                                       'Salem's Lot   
1        2        465                 1 000 Places to See Before You Die   
2        3        407  13 Little Blue Envelopes (Little Blue Envelope...   
3        4         82  1491: New Revelations of the Americas Before C...   
4        5        125                                               1776   

   num_pages publication_date  publisher_id  
0        594       2005-11-01            93  
1        992       2003-05-22           336  
2        322       2010-12-21           135  
3        541       2006-10-10           309  
4        386       2006-07-04           268  


In [3]:
query = """ 
SELECT *
FROM authors
LIMIT 5;
"""

books = pd.io.sql.read_sql(query, con = engine)
print(books)

   author_id                          author
0          1                      A.S. Byatt
1          2  Aesop/Laura Harris/Laura Gibbs
2          3                 Agatha Christie
3          4                   Alan Brennert
4          5        Alan Moore/David   Lloyd


In [4]:
query = """ 
SELECT *
FROM publishers
LIMIT 5;
"""

books = pd.io.sql.read_sql(query, con = engine)
print(books)

   publisher_id                          publisher
0             1                                Ace
1             2                           Ace Book
2             3                          Ace Books
3             4                      Ace Hardcover
4             5  Addison Wesley Publishing Company


In [5]:
query = """ 
SELECT *
FROM ratings
LIMIT 5;
"""

books = pd.io.sql.read_sql(query, con = engine)
print(books)

   rating_id  book_id       username  rating
0          1        1     ryanfranco       4
1          2        1  grantpatricia       2
2          3        1   brandtandrea       5
3          4        2       lorichen       3
4          5        2    mariokeller       2


In [6]:
query = """ 
SELECT *
FROM reviews
LIMIT 5;
"""

books = pd.io.sql.read_sql(query, con = engine)
print(books)

   review_id  book_id       username  \
0          1        1   brandtandrea   
1          2        1     ryanfranco   
2          3        2       lorichen   
3          4        3  johnsonamanda   
4          5        3    scotttamara   

                                                text  
0  Mention society tell send professor analysis. ...  
1  Foot glass pretty audience hit themselves. Amo...  
2  Listen treat keep worry. Miss husband tax but ...  
3  Finally month interesting blue could nature cu...  
4  Nation purpose heavy give wait song will. List...  


## Consultas, resultados y conclusiones:
1. Encuentra el número de libros publicados después del 1 de enero de 2000.
1. Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.
1. Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).
1. Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.
1. Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

In [7]:
# 1. Número de libros publicados despúes del 1 de enero de 2000
query = """
SELECT COUNT(books)
FROM books
WHERE publication_date >= '2000-01-01';
"""

books_published_after_2000 = pd.io.sql.read_sql(query, con = engine)
print(books_published_after_2000)


   count
0    821


Tenemos 821 libros publicados después del 1o de enero del año 2000

In [8]:
# 2. Número de reseñas de usuarios y Calificación promedio para cada libro.

query = """
WITH ratings_summary AS (
    SELECT
        book_id,
        AVG(rating) AS avg_rating
    FROM ratings
    GROUP BY book_id
),
reviews_count AS (
    SELECT
        book_id,
        COUNT(review_id) AS num_reviews
    FROM reviews
    GROUP BY book_id
)
SELECT
    b.book_id,
    b.title,
    rs.avg_rating,
    rv.num_reviews
FROM books AS b
LEFT JOIN ratings_summary AS rs ON b.book_id = rs.book_id
LEFT JOIN reviews_count AS rv ON b.book_id = rv.book_id
ORDER BY rs.avg_rating DESC;
"""
ave_ratings = pd.io.sql.read_sql(query, con = engine)
# Convertimos número de reviews en entero
ave_ratings.num_reviews = ave_ratings.num_reviews.astype('Int64')
print(ave_ratings)

     book_id                                              title  avg_rating  \
0        518               Pop Goes the Weasel (Alex Cross  #5)        5.00   
1        732  The Ghost Map: The Story of London's Most Terr...        5.00   
2        347  In the Hand of the Goddess (Song of the Liones...        5.00   
3        610                           Tai-Pan (Asian Saga  #2)        5.00   
4        330  How to Be a Domestic Goddess: Baking and the A...        5.00   
..       ...                                                ...         ...   
995      915  The World Is Flat: A Brief History of the Twen...        2.25   
996      371                                              Junky        2.00   
997      316                  His Excellency: George Washington        2.00   
998      202                                      Drowning Ruth        2.00   
999      303                               Harvesting the Heart        1.50   

     num_reviews  
0              2  
1            

En esta tabla podemos ver el Título del libro, su Calificación promedio y el Número de reviews (en entero) que ha tenido.

In [9]:
# 3. Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).

query = """ 
SELECT
    p.publisher,
    COUNT(b.book_id) AS total_books
FROM books AS b
JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY total_books DESC
LIMIT 1;
"""
big_editorials = pd.io.sql.read_sql(query, con = engine)
print(big_editorials)

       publisher  total_books
0  Penguin Books           42


La editorial que más libros ha publicado es Penguin Books con 43 libros de más de 50 págs.

In [10]:
# 4. Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.

query = """ 
WITH ratings_summary AS (
    SELECT
        book_id,
        AVG(rating) AS avg_rating
    FROM ratings
    GROUP BY book_id
),
ratings_counts AS (
    SELECT
        book_id,
        COUNT(rating_id) AS num_ratings
    FROM ratings
    GROUP BY book_id
)
SELECT
    a.author,
    b.title,
    rs.avg_rating,
    rc.num_ratings
FROM books AS b
JOIN authors AS a ON b.author_id = a.author_id
JOIN ratings_summary AS rs ON b.book_id = rs.book_id
JOIN ratings_counts AS rc ON b.book_id = rc.book_id
WHERE rc.num_ratings > 49
ORDER BY rs.avg_rating DESC
LIMIT 1;

"""
best_author = pd.io.sql.read_sql(query, con = engine)
print(best_author)

                       author  \
0  J.K. Rowling/Mary GrandPré   

                                               title  avg_rating  num_ratings  
0  Harry Potter and the Prisoner of Azkaban (Harr...    4.414634           82  


El autor que ha tenido mejor calificación promedio en uno de sus libros son J.K. Rowling/Mary GrandPré con el título Harry Potter and the Prisoner of Azkaban. Este libro tuvo 82 calificaciones.

In [11]:
# 5. Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.
query = """ 
WITH reviews_count AS (
    SELECT
        username,
        COUNT(review_id) AS num_reviews
    FROM reviews
    GROUP BY username
), 
ratings_count AS (
    SELECT
        username,
        COUNT(rating_id) AS num_ratings
    FROM ratings
    GROUP BY username
)
SELECT
    AVG(rev.num_reviews) AS avg_reviews_among_active_users
FROM reviews_count AS rev
JOIN ratings_count AS rat
    ON rev.username = rat.username
WHERE rat.num_ratings > 50;


"""

avg_reviews = pd.io.sql.read_sql(query, con = engine)
print(avg_reviews)

   avg_reviews_among_active_users
0                       24.333333


El número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros es de 24.33 reseñas